## deploy

In [1]:
import pandas as pd
import re
import os
import json
import numpy as np

In [2]:
path_dir = r'C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks'
src_dir = os.path.join(path_dir,'txt files')
dst_tmp_dir = os.path.join(path_dir,'csv files temp')
dst_dir = os.path.join(path_dir,'csv files')

lst_files = os.listdir(src_dir)

In [3]:
iter_tp = 0
for iter_fl in lst_files:
    print(iter_tp,iter_fl)
    iter_tp+=1

0 dialog-babi-candidates.txt
1 dialog-babi-kb-all.txt
2 dialog-babi-task1-API-calls-dev.txt
3 dialog-babi-task1-API-calls-trn.txt
4 dialog-babi-task1-API-calls-tst-OOV.txt
5 dialog-babi-task1-API-calls-tst.txt
6 dialog-babi-task2-API-refine-dev.txt
7 dialog-babi-task2-API-refine-trn.txt
8 dialog-babi-task2-API-refine-tst-OOV.txt
9 dialog-babi-task2-API-refine-tst.txt
10 dialog-babi-task3-options-dev.txt
11 dialog-babi-task3-options-trn.txt
12 dialog-babi-task3-options-tst-OOV.txt
13 dialog-babi-task3-options-tst.txt
14 dialog-babi-task4-phone-address-dev.txt
15 dialog-babi-task4-phone-address-trn.txt
16 dialog-babi-task4-phone-address-tst-OOV.txt
17 dialog-babi-task4-phone-address-tst.txt
18 dialog-babi-task5-full-dialogs-dev.txt
19 dialog-babi-task5-full-dialogs-trn.txt
20 dialog-babi-task5-full-dialogs-tst-OOV.txt
21 dialog-babi-task5-full-dialogs-tst.txt
22 dialog-babi-task6-dstc2-candidates.txt
23 dialog-babi-task6-dstc2-dev.txt
24 dialog-babi-task6-dstc2-kb.txt
25 dialog-babi-task

## task 3

In [4]:
fl_no = 14
path_final = os.path.join(src_dir,lst_files[fl_no])
print(path_final)

C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task4-phone-address-dev.txt


In [39]:
df = pd.DataFrame([])
path_to_use = path_final
ct = 0
with open(path_to_use) as topo_file:
    cnt = 0
    cnt_sub = 0
    ar_3,ar_4 = [],[]
    ar_1,ar_2 = [],[]
    ar_6 = []
    ar_7 = {}

    for line in topo_file:
        #if ct>250: break
        ct+=1

        line = line.strip()    
        if not line.strip(): continue

        init_num = re.findall(r'\d+',line)        

        if init_num[0]=='1':

            ar_5 = cnt
            cnt+=1

            if len(ar_3)==0: continue
            ar_6 = np.unique(np.array(ar_6)).tolist()
            ar_6 = " ".join(ar_6)
            ar_6 = '<KB> ' + ar_6 
            ar_6 = [ar_6]*(len(ar_3)-1)
            ar_6.insert(0,'')          

            ar_7 = ['<' + str(i) + '> ' + str(ar_7[i]) + '' for i in ar_7]            
            ar_6 = np.unique(np.array(ar_6)).tolist()
            ar_7 = " ".join(ar_7)
            ar_7 = ' ' + ar_7 + ' '
            ar_7 = [ar_7]*(len(ar_3)-1)
            ar_7.insert(0,'')          

            #print(np.unique(np.array(ar_6)))
            data = list(
                zip(                 
                    ar_3,
                    ar_4,                    
                    [ar_5]*len(ar_3),
                    ar_6,
                    ar_7
                    ))

            df = df.append(data,ignore_index=True)
            cnt_sub = 0
            ar_3,ar_4,ar_6,ar_7 = [],[],[],{}
            continue

        line_splits = line[len(str(init_num[0]))+1:].split('\t')
        #print(line)
        #print(len(line_splits),line_splits,line[2:].split(' '))
        if len(line_splits)!=2:
            line_splits = line[len(str(init_num[0]))+1:].split(' ')            
            line_splits = list(filter(None, line_splits))
            ar_6.append(line_splits[0])
            ar_7[line_splits[1]] = line_splits[2]
            #print(line_splits)
            continue        
        ar_3.append(line_splits[0])
        ar_4.append(line_splits[1])
        cnt_sub+=1

df = df.rename(columns={
     0: 'Call',
     1: 'Response',     
     2: 'Count',
     3: 'KB',
     4: 'Details'


     })
print(df.shape)
fl_dst = lst_files[fl_no][:-3] + 'csv'
path_dst_tmp = os.path.join(dst_tmp_dir,fl_dst)
df.to_csv(path_dst_tmp,index=False)

(1998, 5)


In [40]:
df.head()

,Call,Response,Count,KB,Details
0,hello,hello what can i help you with today,1,,
1,may i have a table at resto_paris_cheap_spanis...,great let me do the reservation,1,<KB> resto_paris_cheap_spanish_7stars,<R_cuisine> spanish <R_address> resto_paris_c...
2,good morning,hello what can i help you with today,2,,
3,may i have a table at resto_madrid_moderate_fr...,great let me do the reservation,2,<KB> resto_madrid_moderate_french_5stars,<R_cuisine> french <R_address> resto_madrid_m...
4,hi,hello what can i help you with today,3,,


In [42]:
prev_count = 0
txt = []
for i in range(len(df)):    
    if df.iloc[i][3]!=prev_count:
        prev_count = df.iloc[i][3]
        txt.append('')
        continue
    txt.append('<context>')  
df['TAG'] = txt

In [43]:
TAGS = [

    '<profile> ',
    ' <context>',
    ' <user> ',
    ' <system> ',
    '<query> ']

c = 0
mn_ar_1 = []
mn_ar_2 = []
mn_ar_3 = []
prev_count = 0
prev_string = ''
for i in range(len(df)):    
    #if c > 100: break
    c+=1    

    if df.iloc[i][3]!=prev_count:
        prev_string = ''
        prev_count = df.iloc[i][3]
    
    a2 = df.iloc[i][0]
    a3 = df.iloc[i][1]    
    if prev_string=='':
            call_string = prev_string + TAGS[4]+ a2    
    else:
        call_string = prev_string +' '+TAGS[4]+ a2  
    prev_string = prev_string + TAGS[2]+  a2 + ' ' + TAGS[3] + a3
    mn_ar_1.append(call_string)
    mn_ar_2.append(a3)

ndf = pd.DataFrame(            
            list(zip(
                mn_ar_1,mn_ar_2                  
                )),

            columns =[
                'Call',
                'Response'
                ]
                ) 
ndf['Call'] = df['TAG'] + ndf['Call']
ndf['Call'] = df['KB'] + df['Details'] + ndf['Call']

print(ndf.shape)
fl_dst = lst_files[fl_no][:-3] + '.csv'
path_dst = os.path.join(dst_dir,fl_dst)
ndf.to_csv(path_dst,index=False)
    

(1998, 2)


In [41]:
ndf.shape

(9969, 2)

In [38]:
ndf.head(10)

,Call,Response
0,<query> hello,hello what can i help you with today
1,<KB> resto_paris_cheap_spanish_7stars<R_cuisin...,great let me do the reservation
2,<query> good morning,hello what can i help you with today
3,<KB> resto_madrid_moderate_french_5stars<R_cui...,great let me do the reservation
4,<query> hi,hello what can i help you with today
5,<KB> resto_bombay_cheap_french_5stars<R_cuisin...,great let me do the reservation
6,<query> good morning,hello what can i help you with today
7,<KB> resto_rome_expensive_spanish_8stars<R_cui...,great let me do the reservation
8,<query> hi,hello what can i help you with today
9,<KB> resto_bombay_moderate_british_1stars<R_cu...,great let me do the reservation


(5009, 2)


## loop define

In [46]:
def convert_to_df(path_final):
    df = pd.DataFrame([])
    path_to_use = path_final
    ct = 0
    with open(path_to_use) as topo_file:
        cnt = 0
        cnt_sub = 0
        ar_3,ar_4 = [],[]
        ar_1,ar_2 = [],[]
        ar_6 = []
        ar_7 = {}

        for line in topo_file:
            #if ct>250: break
            ct+=1

            line = line.strip()    
            if not line.strip(): continue

            init_num = re.findall(r'\d+',line)        

            if init_num[0]=='1':

                ar_5 = cnt
                cnt+=1

                if len(ar_3)==0: continue
                ar_6 = np.unique(np.array(ar_6)).tolist()
                ar_6 = " ".join(ar_6)
                ar_6 = '<KB> ' + ar_6 
                ar_6 = [ar_6]*(len(ar_3)-1)
                ar_6.insert(0,'')          

                ar_7 = ['<' + str(i) + '> ' + str(ar_7[i]) + '' for i in ar_7]            
                ar_6 = np.unique(np.array(ar_6)).tolist()
                ar_7 = " ".join(ar_7)
                ar_7 = ' ' + ar_7 + ' '
                ar_7 = [ar_7]*(len(ar_3)-1)
                ar_7.insert(0,'')          

                #print(np.unique(np.array(ar_6)))
                data = list(
                    zip(                 
                        ar_3,
                        ar_4,                    
                        [ar_5]*len(ar_3),
                        ar_6,
                        ar_7
                        ))

                df = df.append(data,ignore_index=True)
                cnt_sub = 0
                ar_3,ar_4,ar_6,ar_7 = [],[],[],{}
                continue

            line_splits = line[len(str(init_num[0]))+1:].split('\t')
            #print(line)
            #print(len(line_splits),line_splits,line[2:].split(' '))
            if len(line_splits)!=2:
                line_splits = line[len(str(init_num[0]))+1:].split(' ')            
                line_splits = list(filter(None, line_splits))
                ar_6.append(line_splits[0])
                ar_7[line_splits[1]] = line_splits[2]
                #print(line_splits)
                continue        
            ar_3.append(line_splits[0])
            ar_4.append(line_splits[1])
            cnt_sub+=1

    df = df.rename(columns={
        0: 'Call',
        1: 'Response',     
        2: 'Count',
        3: 'KB',
        4: 'Details'


        })
    print(df.shape)
    fl_dst = lst_files[fl_no][:-3] + 'csv'
    path_dst_tmp = os.path.join(dst_tmp_dir,fl_dst)
    df.to_csv(path_dst_tmp,index=False)
    return df

TAGS = [

    '<profile> ',
    ' <context>',
    ' <user> ',
    ' <system> ',
    '<query> ']

def add_context(df):
    prev_count = 0
    txt = []
    for i in range(len(df)):    
        if df.iloc[i][2]!=prev_count:
            prev_count = df.iloc[i][2]
            txt.append('')
            continue
        txt.append('<context>')  
    df['TAG'] = txt
    return df



def parse_from_df(df):
    c = 0
    mn_ar_1 = []
    mn_ar_2 = []
    mn_ar_3 = []
    prev_count = 0
    prev_string = ''
    for i in range(len(df)):    
        #if c > 100: break
        c+=1    

        if df.iloc[i][2]!=prev_count:
            prev_string = ''
            prev_count = df.iloc[i][2]
        
        a2 = df.iloc[i][0]
        a3 = df.iloc[i][1]            
        if prev_string=='':
            call_string = prev_string + TAGS[4]+ a2    
        else:
            call_string = prev_string +' '+TAGS[4]+ a2    
        prev_string = prev_string + TAGS[2]+ a2  + TAGS[3] + a3
        mn_ar_1.append(call_string)
        mn_ar_2.append(a3)

    ndf = pd.DataFrame(            
                list(zip(
                    mn_ar_1,mn_ar_2                  
                    )),

                columns =[
                    'Call',
                    'Response'
                    ]
                    ) 

    ndf['Call'] = df['TAG'] + ndf['Call']
    ndf['Call'] = df['KB'] + df['Details'] + ndf['Call']
    print(ndf.shape)
    fl_dst = lst_files[fl_no][:-3] + '.csv'
    path_dst = os.path.join(dst_dir,fl_dst)
    ndf.to_csv(path_dst,index=False)   

In [47]:
for i in range(12,20):
    fl_no = i
    path_final = os.path.join(src_dir,lst_files[fl_no])
    print(i,path_final)

12 C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task3-options-tst-OOV.txt
13 C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task3-options-tst.txt
14 C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task4-phone-address-dev.txt
15 C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task4-phone-address-trn.txt
16 C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task4-phone-address-tst-OOV.txt
17 C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task4-phone-address-tst.txt
18 C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task5-full-dialogs-dev.txt
19 C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task5-full-dialogs-trn.txt


## loop run

In [48]:
for i in range(14,18):
    fl_no = i
    path_final = os.path.join(src_dir,lst_files[fl_no])
    df = convert_to_df(path_final)
    df = add_context(df)
    parse_from_df(df)
    

(1998, 5)
(1998, 2)
(1998, 5)
(1998, 2)
(1998, 5)
(1998, 2)
(1998, 5)
(1998, 2)
